# Pre-processing California Allocation data for WaDEQA upload.
Date Updated: 07/21/2021
Purpose:  To pre-process the California data into one master file for simple DataFrame creation and extraction

Notes:
- asdf

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/California/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
# Input Data & dataframe creation
fileInput1 = "EWRIMS MASTER FLAT FILE DATA DICTIONARY DRAFT 1-17-20.xlsx" # date & beneficial use data
fileInput2 = "POD_Attributes_input.csv" # wr data
fileInput3 = "Points_of_Diversion_20210701_input.csv" # site data

df_emff = pd.read_excel(fileInput1, header=0, sheet_name="ewrims_flat_file", skiprows=0).replace(np.nan, "")
df_poda = pd.read_csv(fileInput2).replace(np.nan, "")
df_point = pd.read_csv(fileInput3).replace(np.nan, "")

In [4]:
# Merging dataframes into one, using left-join.
dfPOD = pd.merge(df_poda, df_point, left_on='CORE_POD_ID', right_on='POD_ID', how='left')
dfPOD = pd.merge(dfPOD, df_emff[['WR_WATER_RIGHT_ID', 'PRIORITY_DATE', 'DIRECT_DIV_SEASON_END', 'DIRECT_DIV_SEASON_START', 'APPLICATION_ACCEPTANCE_DATE', 'USE_CODE']], on='WR_WATER_RIGHT_ID', how='left')
print(len(dfPOD))
dfPOD.head(3)

63900


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,NaT,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,NaT,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,NaT,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic


In [5]:
print("Allocation priority date...")

def formatDateString(inString1):
    inString = str(inString1).strip()
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:
            valD = pd.to_datetime(inString) # Also valD = parse(inString) #--datetuil.parser.parse
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d/%Y')
    except:
        valndf = ''
    return valndf

dfPOD['in_AllocationPriorityDate'] = dfPOD.apply(lambda row: formatDateString(row['PRIORITY_DATE']) if str(row['PRIORITY_DATE']) != ''else formatDateString(row['APPLICATION_ACCEPTANCE_DATE']), axis=1)
dfPOD.head(3)

Allocation priority date...


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,NaT,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,NaT,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,NaT,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,


In [6]:
print("Timeframe start and time frame end...")

def formatDateString2(inString1):
    inString = str(inString1).strip()
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:            
            valD = pd.to_datetime(inString)
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d')
    except:
        valndf = ''
    return valndf

dfPOD['in_AllocationTimeframeStart'] = dfPOD.apply(lambda row: formatDateString2(row['DIRECT_DIV_SEASON_START']), axis=1)
dfPOD['in_AllocationTimeframeEnd'] = dfPOD.apply(lambda row: formatDateString2(row['DIRECT_DIV_SEASON_END']), axis=1)
dfPOD.head(3)

Timeframe start and time frame end...


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,NaT,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,,06/01,10/31
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,NaT,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,,01/01,12/31
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,NaT,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,,04/01,11/01


In [8]:
#Creating WaDE Owner Field.  
#Create from Owner field. If empty, use LastName + FirstName fields.
import re

def retrieveOwner(FN, LN):
    FN = str(FN).strip()
    LN = str(LN).strip()
    outString = FN + " " + LN
    if FN == "" or pd.isnull(FN):
        outString = LN
    if LN == "" or pd.isnull(LN):
        outString = FN
    if (FN == "" and LN == "") or (pd.isnull(FN) and pd.isnull(LN)):
        outString = "Unspecified"
    outString = re.sub("[$@&.;,/\)(-]", "", outString).strip()
    return outString

dfPOD['in_WaDEOwner'] = dfPOD.apply(lambda row: retrieveOwner(row['FIRST_NAME'], row['LAST_NAME']), axis=1)
dfPOD.head(3)

,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd,in_WaDEOwner
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,NaT,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,,06/01,10/31,CARL SHIMEL
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,NaT,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,,01/01,12/31,SHERBURNE SLACK
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,NaT,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,,04/01,11/01,AL RENKER


In [9]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDECA_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = dfPOD['SOURCE_NAME']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A), 'in_WaterSourceNativeID']
    if not (ml.empty):  # check if the series is empty
        outList = ml.iloc[0]
    else:
        outList = ''
    return outList

dfPOD['in_WaterSourceNativeID'] = dfPOD.apply(lambda row: retrieveWaterSourceNativeID( row['SOURCE_NAME']), axis=1)
dfPOD.head(3)

,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd,in_WaDEOwner,in_WaterSourceNativeID
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,NaT,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,,06/01,10/31,CARL SHIMEL,WaDECA_WS1
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,NaT,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,,01/01,12/31,SHERBURNE SLACK,WaDECA_WS2
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,NaT,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,,04/01,11/01,AL RENKER,WaDECA_WS3


## Export Outputs

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfPOD.dtypes)

OID__x                                  int64
CORE_POD_ID                             int64
WR_WATER_RIGHT_ID                       int64
POD_NUMBER                             object
APPL_ID_x                              object
APPL_POD_x                             object
DIRECT_DIV_AMOUNT                      object
DIVERSION_STORAGE_AMOUNT               object
DIVERSION_AC_FT                        object
P_PLACE_ID                              int64
POD_STATUS                             object
FACE_VALUE_AMOUNT                     float64
DIVERSION_TYPE                         object
DIVERSION_CODE_TYPE                    object
WR_TYPE                                object
WR_STATUS                              object
STORAGE_TYPE                           object
POD_UNIT                               object
FIRST_NAME                             object
LAST_NAME                              object
ENTITY_TYPE                            object
OID__y                            

In [11]:
#Exporting to Finished File
dfPOD.to_csv('P_CaliforniaMaster.csv', index=False)  # The output